In [18]:
import random
import pandas as pd

# Constants
NUM_BUSES = 8
NUM_DRIVERS = 10
NUM_8HR_DRIVERS = 5  # Number of 8-hour drivers
NUM_12HR_DRIVERS = NUM_DRIVERS - NUM_8HR_DRIVERS
SHIFT_DURATIONS = {"8hr": 8, "12hr": 12}
DAYS_OF_WEEK = ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday", "Sunday"]
TIME_SLOTS = range(24)  # Each hour of the day

# Genetic Algorithm Parameters
POPULATION_SIZE = 50
MAX_GENERATIONS = 100
MUTATION_RATE = 0.1
CROSSOVER_RATE = 0.8

# Helper Functions
def is_rest_day(driver_id, day_idx):
    """
    Determine if a driver is resting on the given day.
    """
    if driver_id < NUM_8HR_DRIVERS:  # 8-hour drivers rest on weekends
        return DAYS_OF_WEEK[day_idx] in ["Saturday", "Sunday"]
    else:  # 12-hour drivers rest for 2 days after working 1 day
        return day_idx % 3 != 0  # Rest on 2 days out of every 3

# Generate Random Gene
def generate_random_schedule():
    """
    Generate a random valid weekly schedule.
    """
    schedule = []
    for day_idx, day in enumerate(DAYS_OF_WEEK):
        daily_schedule = []
        for driver_id in range(NUM_DRIVERS):
            if is_rest_day(driver_id, day_idx):
                # Rest day
                daily_schedule.append({"driver": driver_id, "bus": None, "start": None, "end": None, "rest": True})
            else:
                # Assign work
                bus = random.randint(0, NUM_BUSES - 1)
                shift_type = "8hr" if driver_id < NUM_8HR_DRIVERS else "12hr"
                start_time = random.randint(6, 10) if shift_type == "8hr" else random.randint(0, 23)
                end_time = (start_time + SHIFT_DURATIONS[shift_type]) % 24
                daily_schedule.append({
                    "driver": driver_id,
                    "bus": bus,
                    "start": start_time,
                    "end": end_time,
                    "rest": False
                })
        schedule.append(daily_schedule)
    return schedule

# Fitness Function
def fitness(schedule):
    """
    Evaluate the fitness of a schedule based on:
    1. Bus coverage: Ensure each bus is assigned a driver for every hour.
    2. Driver constraints: Rest for 8-hour drivers on weekends and for 12-hour drivers after shifts.
    3. Overlap: Avoid overlapping shifts for drivers.
    """
    penalty = 0
    bus_coverage = {day: {hour: [] for hour in TIME_SLOTS} for day in DAYS_OF_WEEK}
    
    for day_idx, daily_schedule in enumerate(schedule):
        for driver_schedule in daily_schedule:
            if driver_schedule["rest"]:
                continue
            bus = driver_schedule["bus"]
            start = driver_schedule["start"]
            end = driver_schedule["end"]
            
            # Coverage
            if bus is not None:
                hours = range(start, end + 1 if end >= start else end + 24)
                for hour in hours:
                    bus_coverage[DAYS_OF_WEEK[day_idx]][hour % 24].append(bus)
            
            # Driver constraints
            if driver_schedule["driver"] < NUM_8HR_DRIVERS and DAYS_OF_WEEK[day_idx] in ["Saturday", "Sunday"]:
                penalty += 10  # Penalize if an 8-hour driver works on weekends.
            if driver_schedule["driver"] >= NUM_8HR_DRIVERS:
                # Check for 12-hour driver rest after working
                if day_idx > 0:
                    prev_day = schedule[day_idx - 1]
                    for prev_driver in prev_day:
                        if prev_driver["driver"] == driver_schedule["driver"] and not prev_driver["rest"]:
                            penalty += 10  # Penalize if rest is violated.
    
    # Check if every bus is covered
    for day, coverage in bus_coverage.items():
        for hour, buses in coverage.items():
            if len(set(buses)) < NUM_BUSES:
                penalty += 5 * (NUM_BUSES - len(set(buses)))  # Penalize uncovered buses.
    
    return -penalty  # Minimize penalty

# Crossover
def crossover(parent1, parent2):
    """
    Single-point crossover between two schedules.
    """
    point = random.randint(0, len(parent1) - 1)
    child = parent1[:point] + parent2[point:]
    return child

# Mutation
def mutate(schedule):
    """
    Mutate a random part of the schedule.
    """
    day_idx = random.randint(0, len(schedule) - 1)
    driver_idx = random.randint(0, NUM_DRIVERS - 1)
    shift_type = "8hr" if driver_idx < NUM_8HR_DRIVERS else "12hr"
    duration = SHIFT_DURATIONS[shift_type]
    bus = random.randint(0, NUM_BUSES - 1)
    start_time = random.randint(6, 10) if shift_type == "8hr" else random.randint(0, 23)
    end_time = (start_time + duration) % 24
    schedule[day_idx][driver_idx] = {
        "driver": driver_idx,
        "bus": bus,
        "start": start_time,
        "end": end_time,
        "rest": False
    }
    return schedule

# Genetic Algorithm
def genetic_algorithm():
    """
    Run the genetic algorithm to find the optimal weekly schedule.
    """
    population = [generate_random_schedule() for _ in range(POPULATION_SIZE)]
    for generation in range(MAX_GENERATIONS):
        population = sorted(population, key=fitness, reverse=True)
        next_generation = population[:5]  # Elitism
        
        while len(next_generation) < POPULATION_SIZE:
            if random.random() < CROSSOVER_RATE:
                parent1, parent2 = random.sample(population[:20], 2)
                child = crossover(parent1, parent2)
            else:
                child = random.choice(population[:20])
            if random.random() < MUTATION_RATE:
                child = mutate(child)
            next_generation.append(child)
        
        population = next_generation
        print(f"Generation {generation + 1}: Best Fitness = {fitness(population[0])}")
    
    return population[0]


# Generate a DataFrame for the schedule with shift information
def generate_schedule_dataframe(schedule, num_8hr_drivers, num_12hr_drivers):
    """
    Convert the genetic algorithm's best schedule into a readable DataFrame with shifts grouped.
    """
    # Define headers for the schedule table
    days = ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday", "Sunday"]
    driver_labels = [f"Driver {d + 1}" for d in range(NUM_DRIVERS)]

    # Assign shifts to drivers
    shifts = ["8hr" if d < num_8hr_drivers else "12hr" for d in range(NUM_DRIVERS)]

    # Initialize DataFrame with drivers as rows and days as columns
    schedule_df = pd.DataFrame(index=driver_labels, columns=["Shift"] + days)

    # Add the shift column
    schedule_df["Shift"] = shifts

    # Populate the DataFrame with the schedule
    for day_idx, daily_schedule in enumerate(schedule):
        for driver_schedule in daily_schedule:
            driver = f"Driver {driver_schedule['driver'] + 1}"
            if driver_schedule["rest"]:
                schedule_df.loc[driver, days[day_idx]] = "Rest"
            else:
                bus = driver_schedule["bus"]
                start = driver_schedule["start"]
                end = driver_schedule["end"]
                schedule_df.loc[driver, days[day_idx]] = f"Bus {bus + 1} ({start:02d}:00 - {end:02d}:00)"

    # Sort by shift for grouping
    schedule_df = schedule_df.sort_values(by="Shift", ascending=False)
    
    # Sort by driver labels
    #schedule_df = schedule_df.sort_index()

    return schedule_df

# Style the schedule for display
def style_schedule(schedule_df):
    """
    Apply styling to the schedule DataFrame for better readability.
    """
    styled_schedule = (
        schedule_df.style.set_properties(
            **{
                "text-align": "center",
                "font-size": "12px",
                "border": "1px solid black",
                "padding": "5px",
            }
        )
        .set_table_styles(
            [
                {"selector": "th", "props": [("background-color", ""), ("font-size", "14px")]},
                {"selector": "td", "props": [("font-size", "12px"), ("padding", "5px")]},
            ]
        )
        .set_caption("Weekly Bus Schedule with Shifts for Drivers")
    )
    return styled_schedule

# Run the Genetic Algorithm
best_schedule = genetic_algorithm()

# Generate and style the DataFrame
schedule_df = generate_schedule_dataframe(best_schedule, num_8hr_drivers=NUM_8HR_DRIVERS, num_12hr_drivers=NUM_12HR_DRIVERS)
styled_schedule = style_schedule(schedule_df)

# Display the styled schedule
styled_schedule




Generation 1: Best Fitness = -4895
Generation 2: Best Fitness = -4895
Generation 3: Best Fitness = -4805
Generation 4: Best Fitness = -4755
Generation 5: Best Fitness = -4700
Generation 6: Best Fitness = -4655
Generation 7: Best Fitness = -4695
Generation 8: Best Fitness = -4670
Generation 9: Best Fitness = -4635
Generation 10: Best Fitness = -4575
Generation 11: Best Fitness = -4455
Generation 12: Best Fitness = -4495
Generation 13: Best Fitness = -4485
Generation 14: Best Fitness = -4465
Generation 15: Best Fitness = -4405
Generation 16: Best Fitness = -4405
Generation 17: Best Fitness = -4470
Generation 18: Best Fitness = -4405
Generation 19: Best Fitness = -4395
Generation 20: Best Fitness = -4430
Generation 21: Best Fitness = -4415
Generation 22: Best Fitness = -4305
Generation 23: Best Fitness = -4310
Generation 24: Best Fitness = -4250
Generation 25: Best Fitness = -4175
Generation 26: Best Fitness = -4175
Generation 27: Best Fitness = -4145
Generation 28: Best Fitness = -4215
G

,Shift,Monday,Tuesday,Wednesday,Thursday,Friday,Saturday,Sunday
Driver 1,8hr,Bus 8 (09:00 - 17:00),Bus 7 (10:00 - 18:00),Bus 7 (07:00 - 15:00),Bus 5 (07:00 - 15:00),Bus 3 (06:00 - 14:00),Bus 3 (08:00 - 16:00),Bus 6 (06:00 - 14:00)
Driver 2,8hr,Bus 8 (07:00 - 15:00),Bus 2 (06:00 - 14:00),Bus 7 (08:00 - 16:00),Bus 5 (08:00 - 16:00),Bus 7 (09:00 - 17:00),Bus 5 (07:00 - 15:00),Bus 5 (08:00 - 16:00)
Driver 3,8hr,Bus 5 (09:00 - 17:00),Bus 2 (07:00 - 15:00),Bus 3 (09:00 - 17:00),Bus 1 (09:00 - 17:00),Bus 1 (08:00 - 16:00),Bus 7 (09:00 - 17:00),Bus 4 (09:00 - 17:00)
Driver 4,8hr,Bus 1 (07:00 - 15:00),Bus 4 (08:00 - 16:00),Bus 8 (08:00 - 16:00),Bus 2 (07:00 - 15:00),Bus 1 (07:00 - 15:00),Bus 6 (07:00 - 15:00),Bus 8 (10:00 - 18:00)
Driver 5,8hr,Bus 2 (08:00 - 16:00),Bus 5 (09:00 - 17:00),Bus 5 (08:00 - 16:00),Bus 7 (07:00 - 15:00),Bus 4 (09:00 - 17:00),Bus 8 (10:00 - 18:00),Bus 7 (10:00 - 18:00)
Driver 6,12hr,Bus 6 (13:00 - 01:00),Bus 8 (09:00 - 21:00),Bus 1 (06:00 - 18:00),Bus 6 (06:00 - 18:00),Bus 5 (20:00 - 08:00),Bus 2 (03:00 - 15:00),Bus 3 (07:00 - 19:00)
Driver 7,12hr,Bus 7 (07:00 - 19:00),Bus 3 (20:00 - 08:00),Bus 6 (17:00 - 05:00),Bus 6 (10:00 - 22:00),Bus 7 (16:00 - 04:00),Bus 6 (01:00 - 13:00),Bus 4 (06:00 - 18:00)
Driver 8,12hr,Bus 2 (07:00 - 19:00),Bus 8 (01:00 - 13:00),Bus 5 (05:00 - 17:00),Bus 4 (03:00 - 15:00),Bus 7 (13:00 - 01:00),Bus 8 (23:00 - 11:00),Bus 3 (13:00 - 01:00)
Driver 9,12hr,Bus 3 (00:00 - 12:00),Bus 7 (20:00 - 08:00),Bus 1 (18:00 - 06:00),Bus 2 (04:00 - 16:00),Bus 6 (05:00 - 17:00),Bus 2 (12:00 - 00:00),Bus 8 (19:00 - 07:00)
Driver 10,12hr,Bus 3 (04:00 - 16:00),Bus 3 (09:00 - 21:00),Bus 8 (01:00 - 13:00),Bus 5 (01:00 - 13:00),Bus 5 (03:00 - 15:00),Bus 7 (05:00 - 17:00),Bus 6 (23:00 - 11:00)
